<a href="https://colab.research.google.com/github/Cecilia-cmd/2024_MLEES/blob/main/Final_Project/project_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Machine Learning Project - Comparing Machine Learning Algorithms for Predicting Biome Type from Soil Microbial Communities

In [ ]:
#---------------- IMPORTATION PACKAGES -------------------------------------
#1.Prepare the data
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

#2. First method PCA
from sklearn.decomposition import PCA
import numpy as np
from sklearn.linear_model import LogisticRegression

#3. Second method KPCA
from sklearn.decomposition import KernelPCA
from scipy import stats
from sklearn.metrics import accuracy_score, f1_score, log_loss
from sklearn.model_selection import cross_val_predict

#Random Forest model
from sklearn.ensemble import RandomForestClassifier

#SVM model
from sklearn.svm import SVC

#KNN
from sklearn.preprocessing import LabelEncoder#because KNN not recognize target labels
from sklearn.neighbors import KNeighborsClassifier
from scipy import stats

#Evaluation
from sklearn.metrics import classification_report

#Visualisation
import matplotlib.pyplot as plt

#Results
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_score


In [ ]:
#---------------------- 1. PREPARE DATA -----------------------------------------
#Load preprocessed data
merged_data = pd.read_csv('/Users/ceciliatorres/Desktop/UNI/ML/Personal ML project/Data/preprocessed_data.csv')

#Define features X and targets y
otu_columns = [col for col in merged_data.columns if col.startswith('OTU_')]
X = merged_data[otu_columns]  # OTU data as features
y = merged_data['biome_clean']  # Target variable

print(X.head())

#Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(y_train.value_counts())

###details for the splitting
num_train_samples = X_train.shape[0]
num_test_samples = X_test.shape[0]
print(f"Number of training samples: {num_train_samples}")
print(f"Number of test samples: {num_test_samples}")


#Standardize the data on the training set only
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # Fit and transform the training data
X_test = scaler.transform(X_test)        # Transform the test data using the same scaler
print(X_train[:5])


In [ ]:
###------------ 2. DIMENSIONALITY REDUCTION FOR FEATURES -----------------------------------
#------------------ A. FIRST METHOD PCA (linear) -------------------------------------------

#Perform PCA without reducing dimensionality
pca = PCA()
pca.fit(X_train)

#Compute the cumulative sum of explained variance ratio
cumsum = np.cumsum(pca.explained_variance_ratio_)

#Find the number of components to preserve 95% variance
d = np.argmax(cumsum >= 0.95) + 1
print(f"Number of components to preserve 95% variance: {d}")

#to visualize the cumulative variance
plt.figure(figsize=(10, 6))
plt.plot(cumsum, linewidth=1.5, color="slateblue", label="Cumulative Variance")
plt.axvline(x=d, color='red', linestyle='-.', linewidth=1.8, label=f'{d} Components (95% Variance)')
plt.xlabel("Number of Principal Components", fontsize=13, labelpad=17)
plt.ylabel("Cumulative Explained Variance", fontsize=13, labelpad=17)
#plt.title("Explained Variance vs. Number of Components", fontsize=16, fontweight="bold", pad=22)
plt.legend(fontsize=12, loc='upper left')
plt.grid(linewidth=0.4, linestyle="--", alpha=0.6)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
#plt.savefig("explained_variance_pca.pdf", format="pdf", bbox_inches="tight")
plt.show()

# Now we can run PCA again with the optimal number of components
pca = PCA(n_components=d)  # d= 422
X_train_PCA = pca.fit_transform(X_train)

#and we apply the same transformation to the test set
X_test_PCA = pca.transform(X_test)

print("Shape of X_train after PCA:", X_train_PCA.shape)
print("Shape of X_test after PCA:", X_test_PCA.shape)


In [ ]:
#--------------------B. SECOND METHOD: KPCA ------------------------

#Test KPCA, (we can play a bit with the parameters to check the changement in X_train_kpca)
kpca = KernelPCA(n_components=2, kernel="rbf", gamma=0.001, fit_inverse_transform=True)
X_train_kpca = kpca.fit_transform(X_train)

print(f"Shape of X_train_kpca: {X_train_kpca.shape}")
print(X_train_kpca[:5])


#Now how to choose the best number of components for KPCA?

#we define the range of components to test
components = range(1, 41)  #test from 1 to 41 components

#we initialize an empty list to store the reconstruction errors
reconstruction_errors = []

#loop through the range of components
for n in components:

    kpca = KernelPCA(n_components=n, kernel="rbf", gamma=0.001, fit_inverse_transform=True)
    X_train_kpca = kpca.fit_transform(X_train)
    X_train_reconstructed = kpca.inverse_transform(X_train_kpca)

    #we calculate the reconstruction error
    error = mean_squared_error(X_train, X_train_reconstructed)
    reconstruction_errors.append(error)

plt.figure(figsize=(8, 6))
plt.plot(components, reconstruction_errors, marker='o')
plt.title("Reconstruction Error vs Number of Components (RBF Kernel)", fontsize=14)
plt.xlabel("Number of Components", fontsize=12)
plt.ylabel("Reconstruction Error (MSE)", fontsize=12)
plt.xticks(components)
plt.grid()
plt.show()

#we can observe on the figure that 20 number of components might be the best choice


#scoring function
def reconstruction_error_scorer(estimator, X):
    X_transformed = estimator.transform(X)
    X_reconstructed = estimator.inverse_transform(X_transformed)
    return -mean_squared_error(X, X_reconstructed)  #negative because GridSearchCV maximizes score

'''
The following sections document the parameter tuning experiments I conducted.
These tests aimed to optimize the `gamma` parameter and the number of components (`n_components`)
for the KernelPCA model using a systematic grid search approach.

--- First Experiment: Tuning `gamma` within a smaller range ---
Initial parameter grid to narrow down the best `gamma`:
Here, I refined the search range based on a previous experiment where `gamma = 0.001` performed well.

param_grid = {
    'gamma': np.linspace(0.0001, 0.001, 10),  # Test gamma values between 0.0001 and 0.001.
    'n_components': [20]  # Fix the number of components for simplicity.
}

# Initialize KernelPCA
kpca = KernelPCA(kernel='rbf', fit_inverse_transform=True)

# Initialize GridSearchCV with 3-fold cross-validation
grid_search = GridSearchCV(
    estimator=kpca,
    param_grid=param_grid,
    scoring=reconstruction_error_scorer,  # Custom scoring function for reconstruction error
    cv=3,  # 3-fold cross-validation
    verbose=1
)

# Perform the grid search
grid_search.fit(X_train)

# Print the best `gamma` value and its corresponding reconstruction error
print(f"Best Gamma: {grid_search.best_params_['gamma']}")  # Example output: 0.0001
print(f"Best Score (Reconstruction Error): {-grid_search.best_score_}")  # Convert back to positive MSE

--- Second Experiment: Joint tuning of `gamma` and `n_components` ---
Refine `gamma` around the previously identified optimal value (0.0001)
and experiment with multiple values of `n_components`:

param_grid = {
    'gamma': np.linspace(0.00005, 0.0002, 10),  # Fine-tuned range around 0.0001.
    'n_components': [10, 15, 20, 25, 30, 35, 40]  # Test different component numbers.
}

# Initialize KernelPCA
kpca = KernelPCA(kernel='rbf', fit_inverse_transform=True)

# Initialize GridSearchCV with 3-fold cross-validation
grid_search = GridSearchCV(
    estimator=kpca,
    param_grid=param_grid,
    scoring=reconstruction_error_scorer,
    cv=3,  # 3-fold cross-validation
    verbose=1,
    n_jobs=-1  # Utilize all available cores for computation
)

# Fit the grid search on training data
grid_search.fit(X_train)

# Print the best parameters and corresponding reconstruction error
print(f"Best Parameters: {grid_search.best_params_}")  # Example: {'gamma': 6.67e-05, 'n_components': 40}
print(f"Best Score (Reconstruction Error): {-grid_search.best_score_}")  # Example: 0.9518

'''

##now we can fine tune the model one last time :

param_grid = {
    'gamma': np.linspace(6.5e-05, 7.5e-05, 10),
    'n_components': range(35, 45, 2)
}


#initialization of the KernelPCA estimator
kpca = KernelPCA(kernel='rbf', fit_inverse_transform=True)

#initialization ofthe GridSearchCV
grid_search = GridSearchCV(
    estimator=kpca,
    param_grid=param_grid,
    scoring=reconstruction_error_scorer,
    cv=10,
    verbose=1,
    n_jobs=-1
)

#fit grid search on training data
grid_search.fit(X_train)

print(f"Best Parameters: {grid_search.best_params_}") #Best Parameters: {'gamma': 6.5e-05, 'n_components': 43}
print(f"Best Score (Reconstruction Error): {-grid_search.best_score_}")

best_kpca = KernelPCA(n_components=43, kernel="rbf", gamma=6.5e-05, fit_inverse_transform=True)
X_train_kpca = best_kpca.fit_transform(X_train)
X_test_kpca = best_kpca.transform(X_test)

print("Shape of X_train after KPCA:", X_train_kpca.shape)
print("Shape of X_test after KPCA:", X_test_kpca.shape)


###Visualization for choice of #components KPCA
#we define the best gamma
best_gamma = 6.5e-05

#range of components to test
n_components_range = range(1, 51)

#list to store reconstruction errors
reconstruction_errors = []

#loop over different numbers of components
for n_components in n_components_range:
    # Create and fit KernelPCA with the current number of components
    kpca = KernelPCA(n_components=n_components, kernel="rbf", gamma=best_gamma, fit_inverse_transform=True)
    X_train_kpca = kpca.fit_transform(X_train)
    X_train_reconstructed = kpca.inverse_transform(X_train_kpca)

    #reconstruction error
    error = mean_squared_error(X_train, X_train_reconstructed)
    reconstruction_errors.append(error)

plt.figure(figsize=(10, 6))
plt.plot(n_components_range, reconstruction_errors, color="slateblue", linewidth=1.5, marker='o', linestyle='-', label="Reconstruction Error")
plt.axvline(x=43, color='red', linestyle='-.', linewidth=1.8, label="Best n_components (43)")
plt.xlabel("Number of Components", fontsize=13, labelpad=17)
plt.ylabel("Reconstruction Error", fontsize=13, labelpad=17)
#plt.title(f"Reconstruction Error vs. Number of Components (gamma={best_gamma:.2e})",  fontsize=16, fontweight="bold", pad=22)
plt.legend(fontsize=12)
plt.grid(linewidth=0.4, linestyle="--", alpha=0.6)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
#plt.savefig("Reconstruction_error_KPCA.pdf", format="pdf", bbox_inches="tight")
plt.show()



Now that we have our reduced data with PCA and KPCA we can begin our comparison of machine learning models for each PCA and KPCA.
Let's begin with PCA data.

In [ ]:
# -------------------- A.Softmax Regression -------------------- #

#we define the Softmax Regression model
softmax_reg = LogisticRegression(multi_class="multinomial", solver="lbfgs", max_iter=2000)

#then we define the hyperparameter grid for Softmax Regression
param_grid = {
    "C": [0.01, 0.1, 1, 10, 100]  #regularization strength
}

#GridSearchCV with cross-validation
grid_search = GridSearchCV(softmax_reg, param_grid, cv=3, scoring="accuracy", n_jobs=-1)
grid_search.fit(X_train_PCA, y_train)

#best parameters and cross-validation accuracy
print("Best Softmax Regression parameters:", grid_search.best_params_) #{'C': 10}
print("Best cross-validation accuracy:", grid_search.best_score_)#0.9794216726298831

#we evaluate the best model on the test set
PCA_Softmax_model = grid_search.best_estimator_
PCA_Softmax_predictions = PCA_Softmax_model.predict(X_test_PCA)
PCA_Softmax_accuracy = accuracy_score(y_test, PCA_Softmax_predictions)
print(f"PCA + Softmax Regression Test Set Accuracy: {PCA_Softmax_accuracy:.4f}")
cm = confusion_matrix(y_test, PCA_Softmax_predictions)
print("Confusion Matrix:\n", cm)


##we evaluate the performance metrics on training set :
train_predictions = PCA_Softmax_model.predict(X_train_PCA)
train_probas = PCA_Softmax_model.predict_proba(X_train_PCA)      #class probabilities
train_accuracy = accuracy_score(y_train, train_predictions)
train_f1 = f1_score(y_train, train_predictions, average='weighted')
train_log_loss = log_loss(y_train, train_probas)  #log Loss

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Training F1 Score: {train_f1:.4f}")
print(f"Training Log Loss: {train_log_loss:.4f}")


#we perform cross-validation predictions
y_pred_cv = cross_val_predict(PCA_Softmax_model, X_train_PCA, y_train, cv=3, method='predict')
y_proba_cv = cross_val_predict(PCA_Softmax_model, X_train_PCA, y_train, cv=3, method='predict_proba')

cv_accuracy = accuracy_score(y_train, y_pred_cv)
cv_f1 = f1_score(y_train, y_pred_cv, average='weighted')
cv_log_loss = log_loss(y_train, y_proba_cv)

print(f"Cross-Validation Accuracy: {cv_accuracy:.4f}")
print(f"Cross-Validation F1 Score: {cv_f1:.4f}")
print(f"Cross-Validation Log Loss: {cv_log_loss:.4f}")


#now predictions and probabilities on the test set
test_predictions = PCA_Softmax_model.predict(X_test_PCA)
test_probas = PCA_Softmax_model.predict_proba(X_test_PCA)

test_accuracy = accuracy_score(y_test, test_predictions)
test_f1 = f1_score(y_test, test_predictions, average='weighted')
test_log_loss = log_loss(y_test, test_probas)

print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")
print(f"Test Log Loss: {test_log_loss:.4f}")


# Classification report
print("\nClassification Report:")
print(classification_report(y_test, PCA_Softmax_predictions))


In [ ]:
# -------------------- B.Random Forest Classifier -------------------- #

#we define the Random Forest Classifier model
rf_clf = RandomForestClassifier(random_state=42)

#Then the hyperparameter grid for Random Forest
param_grid = {
    "n_estimators": [50, 100, 200],  #number of trees in the forest
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10]
}

grid_search = GridSearchCV(rf_clf, param_grid, cv=3, scoring="accuracy", n_jobs=-1)
grid_search.fit(X_train_PCA, y_train)

print("Best Random Forest parameters:", grid_search.best_params_) #{'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 50}
print("Best cross-validation accuracy:", grid_search.best_score_) # 0.9027681878099303

#we evaluate the best model on the test set
PCA_RF_model = grid_search.best_estimator_

#we evaluate the best model on the training set
train_predictions_RF = PCA_RF_model.predict(X_train_PCA)
train_probas_RF = PCA_RF_model.predict_proba(X_train_PCA)
train_accuracy_RF = accuracy_score(y_train, train_predictions_RF)
train_f1_RF = f1_score(y_train, train_predictions_RF, average='weighted')
train_log_loss_RF = log_loss(y_train, train_probas_RF)

print(f"[Random Forest - PCA] Training Accuracy: {train_accuracy_RF:.4f}")
print(f"[Random Forest - PCA] Training F1 Score: {train_f1_RF:.4f}")
print(f"[Random Forest - PCA] Training Log Loss: {train_log_loss_RF:.4f}")

#now cross-validation
y_pred_cv_RF = cross_val_predict(PCA_RF_model, X_train_PCA, y_train, cv=3, method='predict')
y_proba_cv_RF = cross_val_predict(PCA_RF_model, X_train_PCA, y_train, cv=3, method='predict_proba')
cv_accuracy_RF = accuracy_score(y_train, y_pred_cv_RF)
cv_f1_RF = f1_score(y_train, y_pred_cv_RF, average='weighted')
cv_log_loss_RF = log_loss(y_train, y_proba_cv_RF)

print(f"[Random Forest - PCA] CV Accuracy: {cv_accuracy_RF:.4f}")
print(f"[Random Forest - PCA] CV F1 Score: {cv_f1_RF:.4f}")
print(f"[Random Forest - PCA] CV Log Loss: {cv_log_loss_RF:.4f}")

#and test set
test_predictions_RF = PCA_RF_model.predict(X_test_PCA)
test_probas_RF = PCA_RF_model.predict_proba(X_test_PCA)
test_accuracy_RF = accuracy_score(y_test, test_predictions_RF)
test_f1_RF = f1_score(y_test, test_predictions_RF, average='weighted')
test_log_loss_RF = log_loss(y_test, test_probas_RF)

print(f"[Random Forest - PCA] Test Accuracy: {test_accuracy_RF:.4f}")
print(f"[Random Forest - PCA] Test F1 Score: {test_f1_RF:.4f}")
print(f"[Random Forest - PCA] Test Log Loss: {test_log_loss_RF:.4f}")


# Classification report
print("\nClassification Report:")
print(classification_report(y_test, PCA_RF_predictions))


In [ ]:
# -------------------- C.KNN -------------------- #

#first encode the target variable (because feature = categorical)
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)  #fit and transform for training set
y_test_encoded = label_encoder.transform(y_test)        #transform for test set

# we define the KNN model
knn = KNeighborsClassifier()

#now the hyperparameter grid for KNN
param_grid = {
    "n_neighbors": [3, 5, 7, 9],      # Number of neighbors
    "weights": ["uniform", "distance"],  # Weighting scheme
    "metric": ["euclidean", "manhattan"]  # Distance metrics
}

#we perform GridSearchCV with cross-validation
grid_search = GridSearchCV(knn, param_grid, cv=3, scoring="accuracy", n_jobs=-1)
grid_search.fit(X_train_PCA, y_train_encoded)

print("Best KNN parameters:", grid_search.best_params_) #{'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}
print("Best cross-validation accuracy:", grid_search.best_score_)#0.7757202937668697

PCA_KNN_model = grid_search.best_estimator_

#we evaluate the best model on the training set
train_predictions_KNN = PCA_KNN_model.predict(X_train_PCA)
train_probas_KNN = PCA_KNN_model.predict_proba(X_train_PCA)
train_accuracy_KNN = accuracy_score(y_train_encoded, train_predictions_KNN)
train_f1_KNN = f1_score(y_train_encoded, train_predictions_KNN, average='weighted')
train_log_loss_KNN = log_loss(y_train_encoded, train_probas_KNN)

print(f"[KNN - PCA] Training Accuracy: {train_accuracy_KNN:.4f}")
print(f"[KNN - PCA] Training F1 Score: {train_f1_KNN:.4f}")
print(f"[KNN - PCA] Training Log Loss: {train_log_loss_KNN:.4f}")

#now cross validation
y_pred_cv_KNN = cross_val_predict(PCA_KNN_model, X_train_PCA, y_train_encoded, cv=3, method='predict')
y_proba_cv_KNN = cross_val_predict(PCA_KNN_model, X_train_PCA, y_train_encoded, cv=3, method='predict_proba')
cv_accuracy_KNN = accuracy_score(y_train_encoded, y_pred_cv_KNN)
cv_f1_KNN = f1_score(y_train_encoded, y_pred_cv_KNN, average='weighted')
cv_log_loss_KNN = log_loss(y_train_encoded, y_proba_cv_KNN)

print(f"[KNN - PCA] CV Accuracy: {cv_accuracy_KNN:.4f}")
print(f"[KNN - PCA] CV F1 Score: {cv_f1_KNN:.4f}")
print(f"[KNN - PCA] CV Log Loss: {cv_log_loss_KNN:.4f}")

#and test set
test_predictions_KNN = PCA_KNN_model.predict(X_test_PCA)
test_probas_KNN = PCA_KNN_model.predict_proba(X_test_PCA)
test_accuracy_KNN = accuracy_score(y_test_encoded, test_predictions_KNN)
test_f1_KNN = f1_score(y_test_encoded, test_predictions_KNN, average='weighted')
test_log_loss_KNN = log_loss(y_test_encoded, test_probas_KNN)

print(f"[KNN - PCA] Test Accuracy: {test_accuracy_KNN:.4f}")
print(f"[KNN - PCA] Test F1 Score: {test_f1_KNN:.4f}")
print(f"[KNN - PCA] Test Log Loss: {test_log_loss_KNN:.4f}")


# Classification report
print("\nClassification Report:")
print(classification_report(y_test_encoded, PCA_KNN_predictions, target_names=label_encoder.classes_))

Now, let's do the second method with our KPCA data.

In [ ]:
###----------------- A. Softmax Regression -------------------------------------------

#define the Softmax Regression model
softmax_reg = LogisticRegression(multi_class="multinomial", solver="lbfgs", max_iter=2000)

#define the hyperparameter grid for Softmax Regression
param_grid = {
    "C": [0.01, 0.1, 1, 10, 100]
}

#GridSearchCV with cross-validation
grid_search = GridSearchCV(softmax_reg, param_grid, cv=3, scoring="accuracy", n_jobs=-1) #{'C': 100}
grid_search.fit(X_train_kpca, y_train)

print("Best Softmax Regression parameters:", grid_search.best_params_)#{'C': 100}
print("Best cross-validation accuracy:", grid_search.best_score_)#0.977549013453853


KPCA_Softmax_model = grid_search.best_estimator_

#we evaluate on the training set
train_predictions_Softmax = KPCA_Softmax_model.predict(X_train_kpca)
train_probas_Softmax = KPCA_Softmax_model.predict_proba(X_train_kpca)
train_accuracy_Softmax = accuracy_score(y_train, train_predictions_Softmax)
train_f1_Softmax = f1_score(y_train, train_predictions_Softmax, average='weighted')
train_log_loss_Softmax = log_loss(y_train, train_probas_Softmax)

print(f"[Softmax - KPCA] Training Accuracy: {train_accuracy_Softmax:.4f}")
print(f"[Softmax - KPCA] Training F1 Score: {train_f1_Softmax:.4f}")
print(f"[Softmax - KPCA] Training Log Loss: {train_log_loss_Softmax:.4f}")

#now cross validation
y_pred_cv_Softmax = cross_val_predict(KPCA_Softmax_model, X_train_kpca, y_train, cv=3, method='predict')
y_proba_cv_Softmax = cross_val_predict(KPCA_Softmax_model, X_train_kpca, y_train, cv=3, method='predict_proba')
cv_accuracy_Softmax = accuracy_score(y_train, y_pred_cv_Softmax)
cv_f1_Softmax = f1_score(y_train, y_pred_cv_Softmax, average='weighted')
cv_log_loss_Softmax = log_loss(y_train, y_proba_cv_Softmax)

print(f"[Softmax - KPCA] CV Accuracy: {cv_accuracy_Softmax:.4f}")
print(f"[Softmax - KPCA] CV F1 Score: {cv_f1_Softmax:.4f}")
print(f"[Softmax - KPCA] CV Log Loss: {cv_log_loss_Softmax:.4f}")

#and test set
test_predictions_Softmax = KPCA_Softmax_model.predict(X_test_kpca)
test_probas_Softmax = KPCA_Softmax_model.predict_proba(X_test_kpca)
test_accuracy_Softmax = accuracy_score(y_test, test_predictions_Softmax)
test_f1_Softmax = f1_score(y_test, test_predictions_Softmax, average='weighted')
test_log_loss_Softmax = log_loss(y_test, test_probas_Softmax)

print(f"[Softmax - KPCA] Test Accuracy: {test_accuracy_Softmax:.4f}")
print(f"[Softmax - KPCA] Test F1 Score: {test_f1_Softmax:.4f}")
print(f"[Softmax - KPCA] Test Log Loss: {test_log_loss_Softmax:.4f}")


# Classification report
print("\nClassification Report:")
print(classification_report(y_test, KPCA_Softmax_predictions))




In [ ]:
#----------------- B. Random Forest  -------------------------------------------


#define the Random Forest Classifier model
rf_clf = RandomForestClassifier(random_state=42)

#now the hyperparameter grid for Random Forest
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10]
}

#GridSearchCV with cross-validation
grid_search = GridSearchCV(rf_clf, param_grid, cv=3, scoring="accuracy", n_jobs=-1)
grid_search.fit(X_train_kpca, y_train)

print("Best Random Forest parameters:", grid_search.best_params_) #{'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
print("Best cross-validation accuracy:", grid_search.best_score_) #0.960716004435796


KPCA_RF_model = grid_search.best_estimator_

#we evaluate on the training set
train_predictions_RF_KPCA = KPCA_RF_model.predict(X_train_kpca)
train_probas_RF_KPCA = KPCA_RF_model.predict_proba(X_train_kpca)
train_accuracy_RF_KPCA = accuracy_score(y_train, train_predictions_RF_KPCA)
train_f1_RF_KPCA = f1_score(y_train, train_predictions_RF_KPCA, average='weighted')
train_log_loss_RF_KPCA = log_loss(y_train, train_probas_RF_KPCA)

print(f"[Random Forest - KPCA] Training Accuracy: {train_accuracy_RF_KPCA:.4f}")
print(f"[Random Forest - KPCA] Training F1 Score: {train_f1_RF_KPCA:.4f}")
print(f"[Random Forest - KPCA] Training Log Loss: {train_log_loss_RF_KPCA:.4f}")

#now crossvalidation
y_pred_cv_RF_KPCA = cross_val_predict(KPCA_RF_model, X_train_kpca, y_train, cv=3, method='predict')
y_proba_cv_RF_KPCA = cross_val_predict(KPCA_RF_model, X_train_kpca, y_train, cv=3, method='predict_proba')
cv_accuracy_RF_KPCA = accuracy_score(y_train, y_pred_cv_RF_KPCA)
cv_f1_RF_KPCA = f1_score(y_train, y_pred_cv_RF_KPCA, average='weighted')
cv_log_loss_RF_KPCA = log_loss(y_train, y_proba_cv_RF_KPCA)

print(f"[Random Forest - KPCA] CV Accuracy: {cv_accuracy_RF_KPCA:.4f}")
print(f"[Random Forest - KPCA] CV F1 Score: {cv_f1_RF_KPCA:.4f}")
print(f"[Random Forest - KPCA] CV Log Loss: {cv_log_loss_RF_KPCA:.4f}")

#and test set
test_predictions_RF_KPCA = KPCA_RF_model.predict(X_test_kpca)
test_probas_RF_KPCA = KPCA_RF_model.predict_proba(X_test_kpca)
test_accuracy_RF_KPCA = accuracy_score(y_test, test_predictions_RF_KPCA)
test_f1_RF_KPCA = f1_score(y_test, test_predictions_RF_KPCA, average='weighted')
test_log_loss_RF_KPCA = log_loss(y_test, test_probas_RF_KPCA)

print(f"[Random Forest - KPCA] Test Accuracy: {test_accuracy_RF_KPCA:.4f}")
print(f"[Random Forest - KPCA] Test F1 Score: {test_f1_RF_KPCA:.4f}")
print(f"[Random Forest - KPCA] Test Log Loss: {test_log_loss_RF_KPCA:.4f}")


# Classification report
print("\nClassification Report:")
print(classification_report(y_test, KPCA_RF_predictions))


In [ ]:

#----------------- C. KNN  -------------------------------------------

#first we encode the target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

#then we define the KNN model
knn = KNeighborsClassifier()

#and the hyperparameter grid for KNN
param_grid = {
    "n_neighbors": [3, 5, 7, 9],      #number of neighbors
    "weights": ["uniform", "distance"],  #weighting scheme
    "metric": ["euclidean", "manhattan"]  #distance metrics
}

#GridSearchCV with cross-validation
grid_search = GridSearchCV(knn, param_grid, cv=3, scoring="accuracy", n_jobs=-1)
grid_search.fit(X_train_kpca, y_train_encoded)

print("Best KNN parameters:", grid_search.best_params_) #{'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}
print("Best cross-validation accuracy:", grid_search.best_score_) #0.9756972778440357


KPCA_KNN_model = grid_search.best_estimator_

#we evaluate on the training set
train_predictions_KNN_KPCA = KPCA_KNN_model.predict(X_train_kpca)
train_probas_KNN_KPCA = KPCA_KNN_model.predict_proba(X_train_kpca)
train_accuracy_KNN_KPCA = accuracy_score(y_train_encoded, train_predictions_KNN_KPCA)
train_f1_KNN_KPCA = f1_score(y_train_encoded, train_predictions_KNN_KPCA, average='weighted')
train_log_loss_KNN_KPCA = log_loss(y_train_encoded, train_probas_KNN_KPCA)

print(f"[KNN - KPCA] Training Accuracy: {train_accuracy_KNN_KPCA:.4f}")
print(f"[KNN - KPCA] Training F1 Score: {train_f1_KNN_KPCA:.4f}")
print(f"[KNN - KPCA] Training Log Loss: {train_log_loss_KNN_KPCA:.4f}")

#now cross validation
y_pred_cv_KNN_KPCA = cross_val_predict(KPCA_KNN_model, X_train_kpca, y_train_encoded, cv=3, method='predict')
y_proba_cv_KNN_KPCA = cross_val_predict(KPCA_KNN_model, X_train_kpca, y_train_encoded, cv=3, method='predict_proba')
cv_accuracy_KNN_KPCA = accuracy_score(y_train_encoded, y_pred_cv_KNN_KPCA)
cv_f1_KNN_KPCA = f1_score(y_train_encoded, y_pred_cv_KNN_KPCA, average='weighted')
cv_log_loss_KNN_KPCA = log_loss(y_train_encoded, y_proba_cv_KNN_KPCA)

print(f"[KNN - KPCA] CV Accuracy: {cv_accuracy_KNN_KPCA:.4f}")
print(f"[KNN - KPCA] CV F1 Score: {cv_f1_KNN_KPCA:.4f}")
print(f"[KNN - KPCA] CV Log Loss: {cv_log_loss_KNN_KPCA:.4f}")

#and testing set
test_predictions_KNN_KPCA = KPCA_KNN_model.predict(X_test_kpca)
test_probas_KNN_KPCA = KPCA_KNN_model.predict_proba(X_test_kpca)
test_accuracy_KNN_KPCA = accuracy_score(y_test_encoded, test_predictions_KNN_KPCA)
test_f1_KNN_KPCA = f1_score(y_test_encoded, test_predictions_KNN_KPCA, average='weighted')
test_log_loss_KNN_KPCA = log_loss(y_test_encoded, test_probas_KNN_KPCA)

print(f"[KNN - KPCA] Test Accuracy: {test_accuracy_KNN_KPCA:.4f}")
print(f"[KNN - KPCA] Test F1 Score: {test_f1_KNN_KPCA:.4f}")
print(f"[KNN - KPCA] Test Log Loss: {test_log_loss_KNN_KPCA:.4f}")


# Classification report
print("\nClassification Report:")
print(classification_report(y_test_encoded, KPCA_KNN_predictions, target_names=label_encoder.classes_))



We executed all models for PCA and KPCA, so now let's compare them with ROC curves.

In [ ]:
#-------------COMPARISON WITH ROC CURVES -------------------------------------


#we binarize the labels for multiclass ROC
y_test_bin = label_binarize(y_test, classes=np.unique(y_test))  #shape: (n_samples, n_classes)
n_classes = y_test_bin.shape[1]

#we create a dictionary to store AUC scores
roc_auc_scores = {}

#function to calculate ROC curves for each model
def plot_roc_curve_multiclass(model, X_test, y_test_bin, label, color):
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        #predict probabilities for the positive class
        y_proba = model.predict_proba(X_test)[:, i]
        fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_proba)
        roc_auc[i] = auc(fpr[i], tpr[i])
    #average AUC across classes
    roc_auc_scores[label] = np.mean(list(roc_auc.values()))
    #plot the micro-average ROC curve
    plt.plot(fpr[1], tpr[1], label=f"{label} (AUC = {roc_auc_scores[label]:.2f})", color=color)

#plot all ROC curves
plt.figure(figsize=(10, 7))

# Softmax Regression
plot_roc_curve_multiclass(PCA_Softmax_model, X_test_PCA, y_test_bin, "Softmax Regression (PCA)", color="slateblue")
plt.plot([0], [0], color="slateblue", linewidth=3, label="_nolegend_")
# Random Forest
plot_roc_curve_multiclass(PCA_RF_model, X_test_PCA, y_test_bin, "Random Forest (PCA)", color="forestgreen")
plt.plot([0], [0], color="forestgreen", linewidth=3, label="_nolegend_")
# KNN
plot_roc_curve_multiclass(PCA_KNN_model, X_test_PCA, y_test_bin, "KNN (PCA)", color="orchid")
plt.plot([0], [0], color="orchid", linewidth=3, label="_nolegend_")
#diagonal line
plt.plot([0, 1], [0, 1], linestyle="--", color="gray", linewidth=1.5)
#plt.title("ROC Curves for Softmax Regression, Random Forest, and KNN (PCA)", fontsize=16, fontweight="bold", pad=22)
plt.xlabel("False Positive Rate", fontsize=13, labelpad=17)
plt.ylabel("True Positive Rate", fontsize=13, labelpad=17)
plt.legend(loc="lower right",fontsize=12)
plt.grid(linewidth=0.4, linestyle="--", alpha=0.6)
plt.tight_layout()
#plt.savefig("ROC curves_PCA.pdf", format="pdf", bbox_inches="tight")
plt.show()

# Print the AUC scores for each model
print("AUC Scores:")
for model_name, auc_score in roc_auc_scores.items():
    print(f"{model_name}: {auc_score:.2f}")


#Evaluation of ROC AUC
#perform 5-fold cross-validation and evaluate multiclass ROC AUC (one-vs-rest strategy) for Softmax REgression
scores_softmax = cross_val_score(PCA_Softmax_model, X_train_PCA, y_train, cv=5, scoring='roc_auc_ovr')

print("Cross-Validation AUC Scores (Multiclass, OvR):", scores_softmax)
print("Mean CV AUC:", scores_softmax.mean())
print("Standard Deviation of CV AUC:", scores_softmax.std())

#cross-validation for Random Forest (PCA)
scores_rf = cross_val_score(PCA_RF_model, X_train_PCA, y_train, cv=5, scoring='roc_auc_ovr')
print("Random Forest Cross-Validation AUC Scores (Multiclass, OvR):", scores_rf)
print("Mean CV AUC (Random Forest):", scores_rf.mean())
print("Standard Deviation of CV AUC (Random Forest):", scores_rf.std())

#cross-validation for KNN (PCA)
scores_knn = cross_val_score(PCA_KNN_model, X_train_PCA, y_train_encoded, cv=5, scoring='roc_auc_ovr')
print("KNN Cross-Validation AUC Scores (Multiclass, OvR):", scores_knn)
print("Mean CV AUC (KNN):", scores_knn.mean())
print("Standard Deviation of CV AUC (KNN):", scores_knn.std())


###ROC curves for KPCA :

#plot all ROC curves
plt.figure(figsize=(10, 7))

# Softmax Regression
plot_roc_curve_multiclass(KPCA_Softmax_model, X_test_kpca, y_test_bin, "Softmax Regression (KPCA)", color="slateblue")
plt.plot([0], [0], color="slateblue", linewidth=3, label="_nolegend_")
# Random Forest
plot_roc_curve_multiclass(KPCA_RF_model, X_test_kpca, y_test_bin, "Random Forest (KPCA)", color="forestgreen")
plt.plot([0], [0], color="forestgreen", linewidth=3, label="_nolegend_")
# KNN
plot_roc_curve_multiclass(KPCA_KNN_model, X_test_kpca, y_test_bin, "KNN (KPCA)", color="orchid")
plt.plot([0], [0], color="orchid", linewidth=3, label="_nolegend_")
#diagonal
plt.plot([0, 1], [0, 1], linestyle="--", color="gray", linewidth=1.5)

#final plot
#plt.title("ROC Curves for Softmax Regression, Random Forest, and KNN (KPCA)", fontsize=16, fontweight="bold", pad=22)
plt.xlabel("False Positive Rate", fontsize=13, labelpad=17)
plt.ylabel("True Positive Rate", fontsize=13, labelpad=17)
plt.legend(loc="lower right",fontsize=12)
plt.grid(linewidth=0.4, linestyle="--", alpha=0.6)
plt.tight_layout()
#plt.savefig("ROC curves_KPCA.pdf", format="pdf", bbox_inches="tight")
plt.show()

# Print the AUC scores for each model
print("AUC Scores:")
for model_name, auc_score in roc_auc_scores.items():
    print(f"{model_name}: {auc_score:.2f}")

